#### Data description:
Aruba:
- 1920-1938
- Daily Pressure & Rainfall

ArubaS:
- 1936-1940
- Daily measurements of various variables measured (sometimes whole table, sometimes only wind)

Bonaire:
- 1920-1941
- Daily rain, max and min pressure and temperature (**not always**)

Curacao:
- 1898-1903
- Hourly wind speed

CuracaoM:
- 1902 - 1919
- Various subdaily

Saba:
- 1920 - 1945
- Subdaily pressure, min and max temperature and daily rain

Statia:
- 1943 - 1946
- Subdaily pressure, daily rain

Willemstad:
- 1912 - 1946
- Various subdaily measurements
- TXT format

#### Notes:
- Lot of variability in time scale (hourly, daily, subdaily)
- Lot of variability in recorded variables (between 1 and 12 variables) -> sparse data if all data is put in one table with all columns


- Values with time vs without time? -> 12 uur voor metingen zonder tijd. Of Tijd kolom die Null is. 
- Lege kolommen? -> mnu
- Suriname kolommen?
- Null vs NaN vs None

In [1]:
import duckdb
import pandas as pd 
import os
from tqdm.auto import tqdm

#### Convert to CSV

In [43]:
# importing pandas as pd 
# read an excel file and convert  
# into a dataframe object 
dirs = ['Aruba', 'Bonaire', 'Curacao', 'Saba', 'Statia']
for dir in tqdm(dirs):
    if not os.path.exists(f'CSVs/{dir}'):
        os.makedirs(f'CSVs/{dir}')
    for file in os.listdir(f'digitized/{dir}'):
        for sheet in ['Regen', 'Overig']:
            try:
                name, ext = file.split('.')
                df = pd.DataFrame(pd.read_excel(f"digitized/{dir}/{file}", sheet_name=sheet)) 
                df.to_csv(f'CSVs/{dir}/{name}_{sheet}.csv')
            except ValueError:
                # Catch sheet not found error from files without all sheets
                print(f"Error processing {file} in {sheet} sheet")
                continue

  0%|          | 0/5 [00:00<?, ?it/s]

Error processing Curacao1901.xlsm in Regen sheet
Error processing Curacao1899.xlsm in Regen sheet
Error processing Curacao1903.xlsm in Regen sheet
Error processing Curacao1902.xlsm in Regen sheet
Error processing Curacao1898.xlsm in Regen sheet
Error processing Curacao1900.xlsm in Regen sheet


In [ ]:
# Special treatment for Suriname (xls files, and no separate sheets for rain and other 
# resulting in a two level header)
if not os.path.exists('CSVs/Suriname'):
    os.makedirs('CSVs/Suriname')
for file in os.listdir('digitized/Suriname'):
    name, ext = file.split('.')
    df = pd.DataFrame(pd.read_excel(f"digitized/Suriname/{file}", header=[0,1])) 
    df.to_csv(f'CSVs/Suriname/{name}.csv')

In [ ]:
# Special treatment for Willemstad (latin1 encoded TXT files following CSV format)
import csv
if not os.path.exists('CSVs/Willemstad'):
    os.makedirs('CSVs/Willemstad')
for file in os.listdir('digitized/Willemstad'):
    name, ext = file.split('.')
    df = pd.read_csv(f"digitized/Willemstad/{file}", delimiter=',', encoding='latin1')
    # use QUOTE_NONE to remove double quotes from the TXT file
    df.to_csv(f'CSVs/Willemstad/{name}.csv', quoting=csv.QUOTE_NONE) 

#### DuckDB Config

In [5]:
%load_ext sql
conn = duckdb.connect('knmi_database.db')
%sql conn --alias duckdb

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

#### Aruba

In [111]:
%%sql
DROP TABLE IF EXISTS Aruba;
CREATE TABLE IF NOT EXISTS Aruba AS
SELECT * FROM
(SELECT * FROM read_csv('CSVs/Aruba/*Regen.csv', union_by_name = true, dateformat = '%Y%m%d'))
FULL JOIN 
(SELECT * FROM read_csv('CSVs/Aruba/*Overig.csv', union_by_name = true, dateformat = '%Y%m%d'))
USING (Stn, Datum)
;
SELECT * FROM Aruba LIMIT 1;

,column0,Stn,Datum,Niveau,Rd,qRd,column0_1,P,qP,column00,...,Ed,qEd,Tw,qTw,D,qD,F,qF,N,qN
0,0,32,1920-01-01,1,0,0,0,764.0,0,<NA>,...,None,<NA>,None,<NA>,None,<NA>,None,<NA>,None,<NA>


In [124]:
cursor = conn.execute("PRAGMA table_info('Aruba');")
columns = [row[1] for row in cursor.fetchall()]
failed_columns = []

for column in columns:
    update_query = f"""
    UPDATE Aruba
    SET {column} = NULL
    WHERE TRIM({column}) = '—';
    """
    try:
        conn.execute(update_query)
    except Exception:
        failed_columns.append(column)
        continue
conn.commit()
print(f"Failed to update columns: {failed_columns}")

Failed to update columns: ['column0', 'Stn', 'Datum', 'Niveau', 'qRd', 'column0_1', 'qP', 'column00', 'Tijd', 'qT', 'qTn', 'qTx', 'qU', 'qEd', 'qTw', 'qD', 'qF', 'qN']


In [167]:
%%sql
df << SELECT column_name, column_type, min, max, approx_unique, count, null_percentage
FROM (SUMMARIZE Aruba)

In [168]:
df.to_latex('Aruba_summary.tex', index=False, float_format="%.2f", escape=False, column_format='lcccccc')

In [126]:
%%sql
SELECT *, datetime AT TIME ZONE 'America/Aruba' AS timestamp FROM
(SELECT *, Datum + to_hours(COALESCE(CAST(Tijd AS INTEGER),24)) AS Datetime FROM Aruba)
LIMIT 1; 

,column0,Stn,Datum,Niveau,Rd,qRd,column0_1,P,qP,column00,...,Tw,qTw,D,qD,F,qF,N,qN,Datetime,timestamp
0,0,32,1920-01-01,1,0,0,0,764.0,0,<NA>,...,None,<NA>,None,<NA>,None,<NA>,None,<NA>,1920-01-02,1920-01-02 04:00:00+00:00


#### Bonaire

In [8]:
%%sql
CREATE TABLE IF NOT EXISTS Bonaire AS
(SELECT *  FROM
  (SELECT * FROM read_csv('CSVs/Bonaire/*Regen.csv', union_by_name = true, dateformat = '%Y%m%d')) t1
FULL JOIN 
  (SELECT * FROM read_csv('CSVs/Bonaire/*Overig.csv', union_by_name = true, dateformat = '%Y%m%d')) t2
USING (Stn, Datum))
;

,Count


In [127]:
cursor = conn.execute("PRAGMA table_info('Bonaire');")
columns = [row[1] for row in cursor.fetchall()]
failed_columns = []

for column in columns:
    update_query = f"""
    UPDATE Bonaire
    SET {column} = NULL
    WHERE TRIM({column}) = '—';
    """
    try:
        conn.execute(update_query)
    except Exception:
        failed_columns.append(column)
        continue
conn.commit()
print(f"Failed to update columns: {failed_columns}")

Failed to update columns: ['column0', 'Stn', 'Datum', 'Niveau', 'qRd', 'column00', 'Tijd', 'Px', 'qPx', 'Pn', 'qPn', 'Tx', 'qTx', 'Tn', 'qTn', 'column0_1', 'qP', 'qT']


In [9]:
%%sql
SELECT *, datetime AT TIME ZONE 'America/Aruba' AS timestamp FROM
(SELECT *, Datum + to_hours(COALESCE(CAST(Tijd AS INTEGER),24)) AS Datetime FROM Bonaire)
LIMIT 1;

,column0,Stn,Datum,Niveau,Rd,qRd,column00,Tijd,Px,qPx,...,qTx,Tn,qTn,column0_1,P,qP,T,qT,Datetime,timestamp
0,0,44,1920-01-01,1,0,0,0,24.0,757.0,0,...,0,28.5,0,<NA>,None,<NA>,None,<NA>,1920-01-02,1920-01-02 04:00:00+00:00


In [128]:
%sql FROM (SUMMARIZE Bonaire);

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column0,BIGINT,0,1097,1278,394.66771191080585,287.04041368132647,162,324,601,20091,0.00
1,Stn,BIGINT,42,49,3,44.14533870887462,2.1860411835670304,42,44,44,20091,0.00
2,Datum,DATE,1920-01-01,1941-12-31,5459,None,None,1930-12-01,1935-02-15,1938-07-26,20091,0.00
3,Niveau,BIGINT,1,9,10,1.143945050022398,1.0614684765908877,1,1,1,20091,0.00
4,Rd,VARCHAR,0,99,291,None,None,None,None,None,20091,6.43
5,qRd,BIGINT,0,7,2,0.4501518092678314,1.717140872020826,0,0,0,20091,0.00
6,column00,BIGINT,0,365,385,182.5,105.79933837222235,91,182,274,20091,98.18
7,Tijd,DOUBLE,8.0,24.0,3,11.90400918403391,4.091752787139371,8.0,14.976716315699367,15.0,20091,43.64
8,Px,DOUBLE,753.0,760.5,14,757.0765027322404,1.2201032108792575,756.3888888888889,757.0,758.0,20091,98.18
9,qPx,BIGINT,0,0,1,0.0,0.0,0,0,0,20091,98.18


#### Curacao

In [11]:
%%sql
CREATE TABLE IF NOT EXISTS Curacao AS
SELECT * FROM
  (SELECT * FROM read_csv('CSVs/Curacao/*Regen.csv', union_by_name = true, dateformat = '%Y%m%d')) t1
FULL JOIN 
  (SELECT * FROM read_csv('CSVs/Curacao/*Overig.csv', union_by_name = true, dateformat = '%Y%m%d')) t2
USING (Stn, Datum)
;

,Count


In [129]:
cursor = conn.execute("PRAGMA table_info('Curacao');")
columns = [row[1] for row in cursor.fetchall()]
failed_columns = []

for column in columns:
    update_query = f"""
    UPDATE Curacao
    SET {column} = NULL
    WHERE TRIM({column}) = '—';
    """
    try:
        conn.execute(update_query)
    except Exception:
        failed_columns.append(column)
        continue
conn.commit()
print(f"Failed to update columns: {failed_columns}")

Failed to update columns: ['column0', 'Stn', 'Datum', 'Niveau', 'qRd', 'column0_1', 'Tijd', 'qF', 'column00', 'qTp', 'qPt', 'qPc', 'qP0', 'qP', 'qT', 'qTw', 'qTd', 'qU', 'qEd', 'qTx', 'qTn', 'qD', 'qFx', 'qDFx', 'qN', 'qC', 'qDn']


In [130]:
%%sql
SELECT *, datetime AT TIME ZONE 'America/Aruba' AS timestamp FROM
(SELECT *, Datum + to_hours(COALESCE(CAST(Tijd AS INTEGER),24)) AS Datetime FROM Curacao)
LIMIT 1;


,column0,Stn,Datum,Niveau,Rd,qRd,column0_1,Tijd,F,qF,...,DFx,qDFx,N,qN,C,qC,Dn,qDn,Datetime,timestamp
0,0,78989,1907-01-01,1,None,7,<NA>,800.0,None,7,...,None,7,None,7,None,7,None,7,1907-02-03 08:00:00,1907-02-03 12:00:00+00:00


In [131]:
%sql FROM (SUMMARIZE Curacao);

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column0,BIGINT,0,365,385,182.02673581942364,105.38346926636754,91,182,273,62419,56.14
1,Stn,BIGINT,78989,78989,1,78989.0,0.0,78989,78989,78989,62419,0.00
2,Datum,DATE,1898-01-01,1919-12-31,7321,None,None,1899-10-13,1901-07-25,1903-04-07,62419,0.00
3,Niveau,BIGINT,1,1,1,1.0,0.0,1,1,1,62419,56.14
4,Rd,VARCHAR,0,T,119,None,None,None,None,None,62419,86.66
5,qRd,BIGINT,0,7,2,4.871288213594361,3.220240269106866,0,7,7,62419,56.14
6,column0_1,BIGINT,0,8759,7828,4379.5,2528.8182191779947,2191,4378,6568,62419,15.79
7,Tijd,DOUBLE,1.0,800.0,32,126.70981592143418,265.6118209818324,7.771365407785331,14.984962855435073,21.998025492323098,62419,0.00
8,F,VARCHAR,0,9.8,41,None,None,None,None,None,62419,32.58
9,qF,BIGINT,0,7,3,2.2830548390714367,3.281238405705358,0,0,7,62419,0.00


#### Saba

In [13]:
%%sql
DROP TABLE IF EXISTS Saba;
CREATE TABLE IF NOT EXISTS Saba AS
(SELECT *  FROM
  (SELECT * FROM read_csv('CSVs/Saba/*Regen.csv', union_by_name = true, dateformat = '%Y%m%d')) t1
FULL JOIN 
  (SELECT * FROM read_csv('CSVs/Saba/*Overig.csv', union_by_name = true, dateformat = '%Y%m%d')) t2
USING (Stn, Datum))
;

,Count
0,38726


In [134]:
cursor = conn.execute("PRAGMA table_info('Saba');")
columns = [row[1] for row in cursor.fetchall()]
failed_columns = []

for column in columns:
    update_query = f"""
    UPDATE Saba
    SET {column} = NULL
    WHERE TRIM({column}) = '—';
    """
    try:
        conn.execute(update_query)
    except Exception:
        failed_columns.append(column)
        continue
conn.commit()
print(f"Failed to update columns: {failed_columns}")

Failed to update columns: ['column0', 'Stn', 'Datum', 'Niveau', 'qRd', 'column0_1', 'Tijd', 'qP', 'qTn', 'qTx', 'qT']


In [132]:
%%sql
SELECT *, datetime AT TIME ZONE 'America/Aruba' AS timestamp FROM
(SELECT *, Datum + to_hours(COALESCE(CAST(Tijd AS INTEGER),24)) AS Datetime FROM Saba)
LIMIT 1;

,column0,Stn,Datum,Niveau,Rd,qRd,column0_1,Tijd,P,qP,Tn,qTn,Tx,qTx,T,qT,Datetime,timestamp
0,0,81,1920-01-01,1,48,0,0,8.0,749,0,—,7,—,7,None,<NA>,1920-01-01 08:00:00,1920-01-01 12:00:00+00:00


In [135]:
%sql FROM (SUMMARIZE Saba);

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column0,BIGINT,0,1097,1278,388.92160305737747,306.99926774092404,146,293,609,38726,0.00
1,Stn,BIGINT,81,83,3,81.5659763466405,0.7892913106056159,81,81,82,38726,0.00
2,Datum,DATE,1920-01-01,1945-12-31,7080,None,None,1929-02-09,1934-05-24,1939-09-12,38726,0.00
3,Niveau,BIGINT,1,1,1,1.0,0.0,1,1,1,38726,0.00
4,Rd,VARCHAR,0,99,370,None,None,None,None,None,38726,20.75
5,qRd,BIGINT,0,7,3,1.4528223932241904,2.838558206621826,0,0,0,38726,0.00
6,column0_1,BIGINT,0,1097,1278,547.5464045782533,316.42226637890207,274,548,822,38726,37.73
7,Tijd,DOUBLE,7.45,24.0,5,13.535755162982028,5.0045606534180855,7.45,13.45,19.45,38726,37.73
8,P,VARCHAR,458,762.8,236,None,None,None,None,None,38726,51.49
9,qP,BIGINT,0,7,2,1.5466533963672555,2.90427044781216,0,0,0,38726,37.73


#### Statia

In [15]:
%%sql
DROP TABLE Statia;
CREATE TABLE IF NOT EXISTS Statia AS
(SELECT *  FROM
  (SELECT * FROM read_csv('CSVs/Statia/*Regen.csv', union_by_name = true, dateformat = '%Y%m%d')) t1
FULL JOIN 
  (SELECT * FROM read_csv('CSVs/Statia/*Overig.csv', union_by_name = true, dateformat = '%Y%m%d')) t2
USING (Stn, Datum))
;

,Count
0,4383


In [138]:
cursor = conn.execute("PRAGMA table_info('Statia');")
columns = [row[1] for row in cursor.fetchall()]
failed_columns = []

for column in columns:
    update_query = f"""
    UPDATE Statia
    SET {column} = NULL
    WHERE TRIM({column}) = '—';
    """
    try:
        conn.execute(update_query)
    except Exception:
        failed_columns.append(column)
        continue
conn.commit()
print(f"Failed to update columns: {failed_columns}")

Failed to update columns: ['column0', 'Stn', 'Datum', 'Niveau', 'qRd', 'column0_1', 'Tijd', 'qP']


In [136]:
%%sql
SELECT *, datetime AT TIME ZONE 'America/Aruba' AS timestamp FROM
(SELECT *, Datum + to_hours(COALESCE(CAST(Tijd AS INTEGER),24)) AS Datetime FROM Statia)
LIMIT 1;

,column0,Stn,Datum,Niveau,Rd,qRd,column0_1,Tijd,P,qP,Datetime,timestamp
0,0,73,1943-01-01,1,0,0,2,16.0,761,0,1943-01-01 16:00:00,1943-01-01 20:00:00+01:00


In [139]:
%sql FROM (SUMMARIZE Statia);

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column0,BIGINT,0,365,385,182.1252566735113,105.4506729078786,91,182,273,4383,0.00
1,Stn,BIGINT,73,73,1,73.0,0.0,73,73,73,4383,0.00
2,Datum,DATE,1943-01-01,1946-12-31,1732,None,None,1944-01-01,1944-12-31,1945-12-31,4383,0.00
3,Niveau,BIGINT,1,2,2,1.000684462696783,0.026156266923742105,1,1,1,4383,0.00
4,Rd,VARCHAR,0,98,169,None,None,None,None,None,4383,4.24
5,qRd,BIGINT,0,7,2,0.29705681040383297,1.4112438904436446,0,0,0,4383,0.00
6,column0_1,BIGINT,0,1097,1278,547.3757700205339,316.35307264087663,274,547,822,4383,0.00
7,Tijd,DOUBLE,8.0,16.0,3,12.0,3.2663589617774624,8.0,12.0,16.0,4383,0.00
8,P,VARCHAR,755,766,13,None,None,None,None,None,4383,56.26
9,qP,BIGINT,0,7,2,3.9383983572895276,3.4728314885410096,0,7,7,4383,0.00


#### Suriname

In [17]:
%%sql 
DROP TABLE IF EXISTS Suriname;
CREATE TABLE IF NOT EXISTS Suriname AS
SELECT
    column00 AS "column00",
    column01 AS "Datum",
    column02 AS "Luchtdruk_8",
    column03 AS "Luchtdruk_12",
    column04 AS "Luchtdruk_18",
    column05 AS "Temp_droog_8",
    column06 AS "Temp_nat_8",
    column07 AS "Temp_verschil_8",
    column08 AS "Temp_droog_12",
    column09 AS "Temp_nat_12",
    column10 AS "Temp_verschil_12",
    column11 AS "Temp_droog_18",
    column12 AS "Temp_nat_18",
    column13 AS "Temp_verschil_18",
    column14 AS "Temp_min",
    column15 AS "Temp_max",
    column16 AS "Dampdruk_8",
    column17 AS "Dampdruk_12",
    column18 AS "Dampdruk_18",
    column19 AS "Rel_vocht_8",
    column20 AS "Rel_vocht_12",
    column21 AS "Rel_vocht_18",
    column22 AS "Wind_richt_8",
    column23 AS "Wind_kracht_8",
    column24 AS "Wind_richt_12",
    column25 AS "Wind_kracht_12",
    column26 AS "Wind_richt_18",
    column27 AS "Wind_kracht_18",
    column28 AS "Bewolking_8",
    column29 AS "Bewolking_12",
    column30 AS "Bewolking_18",
    column31 AS "Regen",
    column32 AS "Regen_wrong",
FROM read_csv('CSVs/Suriname/*.csv', union_by_name = true, dateformat = '%Y%m%d', skip = 2, null_padding = true, ignore_errors = true);

,Count
0,25275


In [149]:
cursor = conn.execute("PRAGMA table_info('Suriname');")
columns = [row[1] for row in cursor.fetchall()]
failed_columns = []

for column in columns:
    update_query = f"""
    UPDATE Suriname
    SET {column} = NULL
    WHERE TRIM({column}) = '-';
    """
    try:
        conn.execute(update_query)
    except Exception:
        failed_columns.append(column)
        continue
conn.commit()
print(f"Failed to update columns: {failed_columns}")

Failed to update columns: ['column00', 'Datum', 'Luchtdruk_8', 'Luchtdruk_12', 'Luchtdruk_18', 'Temp_droog_8', 'Temp_verschil_8', 'Wind_kracht_12', 'Regen']


In [150]:
%%sql
FROM (SUMMARIZE Suriname)

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column00,BIGINT,0,377,406,188.11964391691396,108.90165251974767,94,188,282,25275,0.00
1,Datum,DOUBLE,18960101.0,19621231.0,22883,19290672.9398063,193392.08591152562,19122889.844526347,19290108.98421717,19457541.16936066,25275,3.18
2,Luchtdruk_8,DOUBLE,64.0,1017.2645161290322,940,536.9089220109886,193.98589778492993,603.746400706313,616.664158255227,625.0,25275,5.23
3,Luchtdruk_12,DOUBLE,10.0,1015.7193548387097,927,524.1612515936762,195.98543415819904,588.4832858919547,603.9302531345087,613.134132946707,25275,5.22
4,Luchtdruk_18,DOUBLE,13.0,1014.8129032258064,1261,520.3054917768216,197.34742294744035,585.7055634865461,600.1172114522899,610.8528764680865,25275,5.67
5,Temp_droog_8,DOUBLE,23.480645161290322,1016.058064516129,722,246.79448448074461,50.9395194693209,244.99873335135837,253.9915611814346,261.17428968940726,25275,2.77
6,Temp_nat_8,VARCHAR,1010.47,99.0,944,None,None,None,None,None,25275,5.77
7,Temp_verschil_8,DOUBLE,-37.0,1013.8387096774194,1131,232.86592606244582,85.82453974093721,250.50164919875738,263.64182692307685,272.61146680843206,25275,0.93
8,Temp_droog_12,VARCHAR,183.0,399.0,794,None,None,None,None,None,25275,2.30
9,Temp_nat_12,VARCHAR,166.0,358.0,930,None,None,None,None,None,25275,6.51


#### Willemstad

In [19]:
%%sql
DROP TABLE IF EXISTS Willemstad;
CREATE TABLE IF NOT EXISTS Willemstad AS
SELECT * FROM read_csv('CSVs/Willemstad/*.csv', union_by_name = true, 
dateformat = '%Y%m%d', skip=1,  columns=
{
    'column00': 'BIGINT',
    'Stn': 'BIGINT',
    'Datum': 'DATE',
    'Tijd': 'BIGINT',
    'DD': 'VARCHAR',
    'qDD': 'BIGINT',
    'FK': 'VARCHAR',
    'FF': 'VARCHAR',
    'FF1': 'VARCHAR',
    'FF2': 'VARCHAR',
    'qFF': 'BIGINT',
    'WW': 'VARCHAR',
    'qWW': 'BIGINT',
    'N': 'VARCHAR',
    'qN': 'BIGINT',
    'T': 'VARCHAR',
    'qT': 'BIGINT',
    'U': 'VARCHAR',
    'qU': 'BIGINT',
    'EE': 'VARCHAR',
    'qEE': 'BIGINT',
    'Tw': 'VARCHAR',
    'qTw': 'BIGINT',
    'P': 'VARCHAR',
    'Pt': 'VARCHAR',
    'Pt1': 'VARCHAR',
    'qPt': 'BIGINT',
    'Rd': 'VARCHAR',
    'Dr': 'VARCHAR',
    'qRd': 'BIGINT',
    'EV': 'VARCHAR',
    'qEV': 'BIGINT',
    'Trn': 'VARCHAR',
    'Tin': 'VARCHAR',
    'Tn': 'VARCHAR',
    'qTn': 'BIGINT',
    'Trx': 'VARCHAR',
    'Tix': 'VARCHAR',
    'Tx': 'VARCHAR',
    'qTx': 'BIGINT'
});

,Count
0,58803


In [ ]:
cursor = conn.execute("PRAGMA table_info('Willemstad');")
columns = [row[1] for row in cursor.fetchall()]
failed_columns = []

for column in columns:
    update_query = f"""
    UPDATE Willemstad
    SET {column} = NULL
    WHERE TRIM({column}) = '//////';
    """
    try:
        conn.execute(update_query)
    except Exception:
        failed_columns.append(column)
        continue
conn.commit()
print(f"Failed to update columns: {failed_columns}")

Failed to update columns: ['column00', 'Stn', 'Datum', 'Tijd', 'qDD', 'qFF', 'qWW', 'qN', 'qT', 'qU', 'qEE', 'qTw', 'qPt', 'qRd', 'qEV', 'qTn', 'qTx']


In [27]:
%%sql
UPDATE Willemstad
SET WW = NULL
WHERE TRIM(WW) = '//////////////';

,Count
0,58803


In [28]:
%%sql
FROM (SUMMARIZE SELECT * FROM Willemstad);

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column00,BIGINT,0,2919,2966,826.3362923660358,529.428140989109,397,794,1191,58803,0.00
1,Stn,BIGINT,78989,78989,1,78989.0,0.0,78989,78989,78989,58803,0.00
2,Datum,DATE,1910-01-01,1946-12-31,14032,None,None,1920-01-24,1927-11-17,1937-09-09,58803,0.00
3,Tijd,BIGINT,600,2400,13,1544.6790129755284,598.239128642078,800,1400,1836,58803,0.00
4,DD,VARCHAR,N,ZZW,16,None,None,None,None,None,58803,32.60
5,qDD,BIGINT,0,7,2,2.2821454687686002,3.281312762278335,0,0,7,58803,0.00
6,FK,VARCHAR,0,20,17,None,None,None,None,None,58803,67.70
7,FF,VARCHAR,0,22,16,None,None,None,None,None,58803,81.37
8,FF1,VARCHAR,0,72,60,None,None,None,None,None,58803,91.53
9,FF2,VARCHAR,1,22,20,None,None,None,None,None,58803,92.54


In [31]:
%%sql
FROM (SHOW ALL TABLES);

,database,schema,name,column_names,column_types,temporary
0,knmi_database,main,Aruba,"[Stn, Datum, Niveau, Rd, qRd, P, qP, Tijd, T, ...","[BIGINT, DATE, BIGINT, VARCHAR, BIGINT, VARCHA...",False
1,knmi_database,main,Bonaire,"[column0, Stn, Datum, Niveau, Rd, qRd, column0...","[BIGINT, BIGINT, DATE, BIGINT, VARCHAR, BIGINT...",False
2,knmi_database,main,Curacao,"[column0, Stn, Datum, Niveau, Rd, qRd, column0...","[BIGINT, BIGINT, DATE, BIGINT, VARCHAR, BIGINT...",False
3,knmi_database,main,Saba,"[column0, Stn, Datum, Niveau, Rd, qRd, column0...","[BIGINT, BIGINT, DATE, BIGINT, VARCHAR, BIGINT...",False
4,knmi_database,main,Statia,"[column0, Stn, Datum, Niveau, Rd, qRd, column0...","[BIGINT, BIGINT, DATE, BIGINT, VARCHAR, BIGINT...",False
5,knmi_database,main,Suriname,"[column00, Datum, Luchtdruk_8, Luchtdruk_12, L...","[BIGINT, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBL...",False
6,knmi_database,main,Willemstad,"[column00, Stn, Datum, Tijd, DD, qDD, FK, FF, ...","[BIGINT, BIGINT, DATE, BIGINT, VARCHAR, BIGINT...",False


In [40]:
17537 + 20091 + 62419 + 38726 + 4383 + 25275 + 58803

227234

In [39]:
%%sql
FROM (SUMMARIZE Willemstad)

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column00,BIGINT,0,2919,2966,826.3362923660358,529.428140989109,397,794,1191,58803,0.00
1,Stn,BIGINT,78989,78989,1,78989.0,0.0,78989,78989,78989,58803,0.00
2,Datum,DATE,1910-01-01,1946-12-31,14032,None,None,1920-01-24,1927-11-17,1937-09-09,58803,0.00
3,Tijd,BIGINT,600,2400,13,1544.6790129755284,598.239128642078,800,1400,1836,58803,0.00
4,DD,VARCHAR,N,ZZW,16,None,None,None,None,None,58803,32.60
5,qDD,BIGINT,0,7,2,2.2821454687686002,3.281312762278335,0,0,7,58803,0.00
6,FK,VARCHAR,0,20,17,None,None,None,None,None,58803,67.70
7,FF,VARCHAR,0,22,16,None,None,None,None,None,58803,81.37
8,FF1,VARCHAR,0,72,60,None,None,None,None,None,58803,91.53
9,FF2,VARCHAR,1,22,20,None,None,None,None,None,58803,92.54


#### Join tables

Check that weather stations don't overlap:
- Aruba: 32
- Bonaire: 42, 44, 49
- Curacao: 78989
- Saba: 83, 81, 82
- Statia: 73
- Suriname: 81201
- Willemstad: 78989

In [151]:
%%sql
FROM (SUMMARIZE SELECT * FROM Bonaire)

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column0,BIGINT,0,1097,1278,394.66771191080585,287.04041368132647,162,324,601,20091,0.00
1,Stn,BIGINT,42,49,3,44.14533870887462,2.1860411835670304,42,44,44,20091,0.00
2,Datum,DATE,1920-01-01,1941-12-31,5459,None,None,1930-12-01,1935-02-15,1938-07-26,20091,0.00
3,Niveau,BIGINT,1,9,10,1.143945050022398,1.0614684765908877,1,1,1,20091,0.00
4,Rd,VARCHAR,0,99,291,None,None,None,None,None,20091,6.43
5,qRd,BIGINT,0,7,2,0.4501518092678314,1.717140872020826,0,0,0,20091,0.00
6,column00,BIGINT,0,365,385,182.5,105.79933837222235,91,182,274,20091,98.18
7,Tijd,DOUBLE,8.0,24.0,3,11.90400918403391,4.091752787139371,8.0,14.976716315699367,15.0,20091,43.64
8,Px,DOUBLE,753.0,760.5,14,757.0765027322404,1.2201032108792575,756.3888888888889,757.0,758.0,20091,98.18
9,qPx,BIGINT,0,0,1,0.0,0.0,0,0,0,20091,98.18


In [90]:
%%sql
FROM (DESCRIBE SELECT * FROM Aruba)

,column_name,column_type,null,key,default,extra
0,Stn,BIGINT,YES,None,None,None
1,Datum,DATE,YES,None,None,None
2,Niveau,BIGINT,YES,None,None,None
3,Rd,VARCHAR,YES,None,None,None
4,qRd,BIGINT,YES,None,None,None
5,P,VARCHAR,YES,None,None,None
6,qP,BIGINT,YES,None,None,None
7,Tijd,DOUBLE,YES,None,None,None
8,T,VARCHAR,YES,None,None,None
9,qT,BIGINT,YES,None,None,None


In [100]:
%%sql 
FROM (DESCRIBE Bonaire)

,column_name,column_type,null,key,default,extra
0,column0,BIGINT,YES,None,None,None
1,Stn,BIGINT,YES,None,None,None
2,Datum,DATE,YES,None,None,None
3,Niveau,BIGINT,YES,None,None,None
4,Rd,VARCHAR,YES,None,None,None
5,qRd,BIGINT,YES,None,None,None
6,column00,BIGINT,YES,None,None,None
7,Tijd,DOUBLE,YES,None,None,None
8,Px,DOUBLE,YES,None,None,None
9,qPx,BIGINT,YES,None,None,None


In [97]:
%%sql
FROM (DESCRIBE
SELECT 
    *
FROM
(SELECT * FROM Aruba) AS a
FULL JOIN
(SELECT * FROM Bonaire) AS b
USING (Stn, Datum))

,column_name,column_type,null,key,default,extra
0,Stn,BIGINT,YES,None,None,None
1,Datum,DATE,YES,None,None,None
2,Niveau,BIGINT,YES,None,None,None
3,Rd,VARCHAR,YES,None,None,None
4,qRd,BIGINT,YES,None,None,None
5,P,VARCHAR,YES,None,None,None
6,qP,BIGINT,YES,None,None,None
7,Tijd,DOUBLE,YES,None,None,None
8,T,VARCHAR,YES,None,None,None
9,qT,BIGINT,YES,None,None,None


In [162]:
%%sql
DROP TABLE IF EXISTS Aggregated;
CREATE TABLE Aggregated AS
SELECT 
    Stn,
    Datum,
    COALESCE (a.Niveau, b.Niveau) AS Niveau,
    COALESCE (a.Rd, b.Rd) AS Rd,
    COALESCE (a.qRd, b.qRd) AS qRd,
    COALESCE (a.Tijd, b.Tijd) AS Tijd,
    COALESCE (a.Tx, CAST(b.Tx AS VARCHAR)) AS Tx,
    COALESCE (a.qTx, b.qTx) AS qTx,
    COALESCE (a.Tn, CAST(b.Tn AS VARCHAR)) AS Tn,
    COALESCE (a.qTn, b.qTn) AS qTn,
    COALESCE (a.P, b.P) AS P,
    COALESCE (a.qP, b.qP) AS qP,
    COALESCE (a.T, b.T) AS T,
    COALESCE (a.qT, b.qT) AS qT,
    U, qU, Ed, qEd, Tw, qTw, D, qD, F, qF, N, qN,
    Px, qPx, Pn, qPn
FROM
(SELECT * FROM Aruba) AS a
FULL JOIN
(SELECT * FROM Bonaire) AS b
USING (Stn, Datum)

,Count
0,37628


In [163]:
%%sql
FROM (SUMMARIZE Aggregated)

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,Stn,BIGINT,32,49,4,38.48485170617625,6.265772328680998,32,42,44,37628,0.00
1,Datum,DATE,1920-01-01,1941-12-31,5459,None,None,1932-09-13,1936-09-30,1938-08-03,37628,0.00
2,Niveau,BIGINT,1,9,10,1.076857659189965,0.7789338343146792,1,1,1,37628,0.00
3,Rd,VARCHAR,0,990,345,None,None,None,None,None,37628,21.55
4,qRd,BIGINT,0,7,2,1.5087169129371745,2.878369651256062,0,0,0,37628,0.00
5,Tijd,DOUBLE,7.0,24.0,6,13.47675530066041,5.540707349744609,8.0,14.351517542430297,17.0,37628,38.83
6,Tx,VARCHAR,26.0,34.6,70,None,None,None,None,None,37628,96.24
7,qTx,BIGINT,0,7,2,6.177973129872284,2.2536373088753066,7,7,7,37628,67.95
8,Tn,VARCHAR,20.4,32,117,None,None,None,None,None,37628,96.24
9,qTn,BIGINT,0,7,2,6.177973129872284,2.2536373088753066,7,7,7,37628,67.95


#### Summarized

In [199]:
%%sql 
df << FROM (SUMMARIZE Aruba)

In [200]:
import numpy as np
print(np.average(df['null_percentage']))
print(np.average(df['count']))

44.07620689655171
17537.0


In [201]:
%%sql 
df << FROM (SUMMARIZE Bonaire)

In [202]:
print(np.average(df['null_percentage']))
print(np.average(df['count']))

55.48809523809524
20091.0


In [217]:
%%sql 
df << FROM (SUMMARIZE Curacao);

In [219]:
print(np.average(df['null_percentage']))
print(np.average(df['count']))
df.head(50)

77.88148936170212
62419.0


,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column0,BIGINT,0,365,385,182.02673581942364,105.38346926636754,91,182,273,62419,56.14
1,Stn,BIGINT,78989,78989,1,78989.0,0.0,78989,78989,78989,62419,0.00
2,Datum,DATE,1898-01-01,1919-12-31,7321,None,None,1899-10-13,1901-07-25,1903-04-07,62419,0.00
3,Niveau,BIGINT,1,1,1,1.0,0.0,1,1,1,62419,56.14
4,Rd,VARCHAR,0,T,119,None,None,None,None,None,62419,86.66
5,qRd,BIGINT,0,7,2,4.871288213594361,3.220240269106866,0,7,7,62419,56.14
6,column0_1,BIGINT,0,8759,7828,4379.5,2528.8182191779947,2191,4378,6568,62419,15.79
7,Tijd,DOUBLE,1.0,800.0,32,126.70981592143418,265.6118209818324,7.771365407785331,14.984962855435073,21.998025492323098,62419,0.00
8,F,VARCHAR,0,9.8,41,None,None,None,None,None,62419,32.58
9,qF,BIGINT,0,7,3,2.2830548390714367,3.281238405705358,0,0,7,62419,0.00


In [205]:
%%sql 
df << FROM (SUMMARIZE Saba)

In [206]:
print(np.average(df['null_percentage']))
print(np.average(df['count']))

41.98374999999999
38726.0


In [207]:
%%sql 
df << FROM (SUMMARIZE Statia)

In [208]:
print(np.average(df['null_percentage']))
print(np.average(df['count']))

6.05
4383.0


In [209]:
%%sql 
df << FROM (SUMMARIZE Suriname)

In [210]:
print(np.average(df['null_percentage']))
print(np.average(df['count']))

21.586060606060606
25275.0


In [211]:
%%sql 
df << FROM (SUMMARIZE Willemstad)

In [212]:
print(np.average(df['null_percentage']))
print(np.average(df['count']))

39.8865
58803.0


Aruba & 17537 & 44.1\
Bonaire & 20091 & 55.5\
Curacao & 62419 & 77.9\
Saba & 38726 & 42.0\
St. Eustatius & 4383 & 6.1\
Suriname & 25275 & 21.6\
Willemstad & 58803 & 39.9